# Rocky Wu - jywu3

As usual, import the packages we want

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
import bqplot

### Explore the dataset:

The name of the dataset: Yellow Taxi trip data 2021-01 in NYC

We can obtain the dataset at the TLC official site. [link towards the dataset webpage](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

The link towards the dataset:
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.csv

Preferably, download the dataset first as the dataset is fairly large, 126MB. 

No identification of the license, so I am assuming we can play around with it. We do have an user guide of the dataset, but not much was mentioned on what we can do with the data. Link to the user guide: https://www1.nyc.gov/assets/tlc/downloads/pdf/trip_record_user_guide.pdf

The dictionary of the dataset:https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf



In [2]:
# Original import using the link
# data = pd.read_csv("https://s3.amazonaws.com/
# nyc-tlc/trip+data/yellow_tripdata_2021-01.csv")
data = pd.read_csv("../data/yellow_tripdata_2021-01.csv", 
                   low_memory=False)

There was a warning indicating the dtype conflicts in some of the columns in the dataset. Will deal with the warning in the later stage. Escaping the warning by setting low_memory = False

Take a look of the dataset

In [3]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5


In total 1369765 rows inside the dataset, indeed a large one.

In [4]:
len(data)

1369765

How many columns:

In [5]:
print(len(data.columns))

18


Take a look on the columns

In [6]:
for c in data.columns:
    print(c)

VendorID
tpep_pickup_datetime
tpep_dropoff_datetime
passenger_count
trip_distance
RatecodeID
store_and_fwd_flag
PULocationID
DOLocationID
payment_type
fare_amount
extra
mta_tax
tip_amount
tolls_amount
improvement_surcharge
total_amount
congestion_surcharge


### Explore the values inside each column

The below prints out if there were any nan values inside each column, before printing out the nan values, we would need to convert some columns' datatype

In [7]:
data.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

From the above, we can tell that the two datetime columns and the flag column need some cleaning. We would convert the object in datetime columns into datetime objectsm and the flag should be converted to string/text data type. 

The two datetime columns first:

In [8]:
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])

In [9]:
data.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                    float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [10]:
data['tpep_pickup_datetime'].head(), data['tpep_dropoff_datetime'].head()

(0   2021-01-01 00:30:10
 1   2021-01-01 00:51:20
 2   2021-01-01 00:43:30
 3   2021-01-01 00:15:48
 4   2021-01-01 00:31:49
 Name: tpep_pickup_datetime, dtype: datetime64[ns],
 0   2021-01-01 00:36:12
 1   2021-01-01 00:52:19
 2   2021-01-01 01:11:06
 3   2021-01-01 00:31:01
 4   2021-01-01 00:48:21
 Name: tpep_dropoff_datetime, dtype: datetime64[ns])

Good, now we have changed the datatype of the dates into the correct type, lets see if there are any nan or strange entries in the column

In [11]:
pd.to_datetime(data['tpep_pickup_datetime']).dt.date.unique()

array([datetime.date(2021, 1, 1), datetime.date(2020, 12, 31),
       datetime.date(2021, 1, 2), datetime.date(2021, 1, 4),
       datetime.date(2021, 1, 3), datetime.date(2021, 1, 5),
       datetime.date(2021, 1, 6), datetime.date(2021, 1, 7),
       datetime.date(2021, 1, 8), datetime.date(2021, 1, 9),
       datetime.date(2021, 1, 10), datetime.date(2021, 1, 11),
       datetime.date(2021, 2, 1), datetime.date(2021, 2, 22),
       datetime.date(2021, 1, 12), datetime.date(2021, 1, 13),
       datetime.date(2021, 1, 14), datetime.date(2009, 1, 1),
       datetime.date(2021, 1, 15), datetime.date(2021, 1, 16),
       datetime.date(2021, 1, 17), datetime.date(2021, 1, 18),
       datetime.date(2021, 1, 19), datetime.date(2021, 1, 20),
       datetime.date(2021, 1, 21), datetime.date(2021, 1, 22),
       datetime.date(2021, 1, 23), datetime.date(2021, 1, 24),
       datetime.date(2021, 1, 25), datetime.date(2021, 1, 26),
       datetime.date(2008, 12, 31), datetime.date(2021, 1, 27),
 

Notice that we have three errorous data in the dataset, one indicated 2008/12/31, one indicated 2020/12/31 and one indicating 2009/12/31. We should remove these entries.

In [12]:
data[data['tpep_pickup_datetime'] <= "2020-12-31 23:59:59"]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
455,2.0,2020-12-31 23:59:06,2021-01-01 00:02:51,2.0,0.74,1.0,N,237,236,1.0,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5
530,2.0,2020-12-31 21:40:20,2020-12-31 22:16:09,1.0,17.96,1.0,N,249,213,2.0,49.5,0.5,0.5,0.00,0.0,0.3,53.30,2.5
546,2.0,2020-12-31 23:57:17,2021-01-01 00:17:40,1.0,3.80,1.0,N,163,166,1.0,16.0,0.5,0.5,3.96,0.0,0.3,23.76,2.5
637,2.0,2020-12-31 18:41:27,2021-01-01 17:52:03,1.0,2.61,1.0,N,90,229,1.0,10.0,0.5,0.5,2.76,0.0,0.3,16.56,2.5
902,2.0,2020-12-31 23:55:40,2021-01-01 00:24:51,1.0,6.45,1.0,N,231,17,2.0,24.5,0.5,0.5,0.00,0.0,0.3,28.30,2.5
936,2.0,2020-12-31 13:44:02,2020-12-31 13:53:43,1.0,3.76,1.0,N,170,226,1.0,12.5,0.5,0.5,4.08,0.0,0.3,20.38,2.5
1043,2.0,2020-12-31 23:59:40,2021-01-01 00:25:51,1.0,12.73,1.0,N,43,200,2.0,36.5,0.5,0.5,0.00,2.8,0.3,43.10,2.5
1661,2.0,2020-12-31 21:35:59,2020-12-31 21:38:20,1.0,0.66,1.0,N,75,74,2.0,4.0,0.5,0.5,0.00,0.0,0.3,5.30,0.0
1662,2.0,2020-12-31 21:43:40,2020-12-31 21:52:26,1.0,2.15,1.0,N,263,74,2.0,9.0,0.5,0.5,0.00,0.0,0.3,12.80,2.5
1880,2.0,2020-12-31 23:31:29,2020-12-31 23:37:51,1.0,1.57,1.0,N,137,79,1.0,7.5,0.5,0.5,2.82,0.0,0.3,14.12,2.5


In [13]:
dropIndex = list(data[data['tpep_pickup_datetime'] <= "2020-12-31 23:59:59"].index)

In [14]:
dropIndex[:5]

[455, 530, 546, 637, 902]

In [15]:
data.drop([i for i in dropIndex], inplace=True)

In [16]:
pd.to_datetime(data['tpep_pickup_datetime']).dt.date.unique()

array([datetime.date(2021, 1, 1), datetime.date(2021, 1, 2),
       datetime.date(2021, 1, 4), datetime.date(2021, 1, 3),
       datetime.date(2021, 1, 5), datetime.date(2021, 1, 6),
       datetime.date(2021, 1, 7), datetime.date(2021, 1, 8),
       datetime.date(2021, 1, 9), datetime.date(2021, 1, 10),
       datetime.date(2021, 1, 11), datetime.date(2021, 2, 1),
       datetime.date(2021, 2, 22), datetime.date(2021, 1, 12),
       datetime.date(2021, 1, 13), datetime.date(2021, 1, 14),
       datetime.date(2021, 1, 15), datetime.date(2021, 1, 16),
       datetime.date(2021, 1, 17), datetime.date(2021, 1, 18),
       datetime.date(2021, 1, 19), datetime.date(2021, 1, 20),
       datetime.date(2021, 1, 21), datetime.date(2021, 1, 22),
       datetime.date(2021, 1, 23), datetime.date(2021, 1, 24),
       datetime.date(2021, 1, 25), datetime.date(2021, 1, 26),
       datetime.date(2021, 1, 27), datetime.date(2021, 1, 28),
       datetime.date(2021, 1, 29), datetime.date(2021, 1, 30),
  

Good, the errorous data were all dropped. Same procedure is taken for the tpep_dropoff_datetime column

Now the flag one is bit complicated, we would have to deal with nan values...

Actually, we would not need to convert the flag columns, as we still can filter out the results using string. But we would still need to cope with the nan values in this column.

In [17]:
data['store_and_fwd_flag'] = data['store_and_fwd_flag'].astype(str)

In [18]:
data[data['store_and_fwd_flag'] == 'N'].head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5


After cleaning the datatypes, we can use the pd.isnull to determine if there were nan values inside the columns

Print out the if nan values exists in the column

In [19]:
for c in data.columns:
    if pd.isnull(data[c].unique()).any():
        print(c)
    # print(c, pd.isnull(data[c].unique()).any())

VendorID
passenger_count
RatecodeID
payment_type


From the output above, we now know that there were some nan values in VendorID, passenger_count, RatecodeID, payment_type. Not really a huge problem, but we would need to careful when creating viz. Good news is that we do not have any nan values inside our total_pay and other numerical columns

### Summary of the dataset:

There were in total 18 columns in the dataset. The names were printed above alongside their datatypes. The dataset represents the total trips of yellow taxi at NYC during January, 2021. The dataset includes many fields such as DateTime for each trip, specified to pick up time and drop off time. The trip distance, the fares, the rates, and the payment types and passenger numbers for each entry. On the site, the raw data is collected within the partnership with the NYC DOITT, but I could not find the person who created these datasets. The credits are not given either on the website or in the dataset. This dataset is interesting as we can analyze the pay for each driver and the relationship of their pay with the pay standard we have in NYC. Also, we can tell from this dataset that we could use this dataset to find out the market of taxis in NYC. The further analysis that I described above would also be interesting to explore.

### Plot of interest:

Lets make a simple one, a frequency graph with the x axis as everyday in January, and the y-axis as the count of taxi drives each date. (For each date, we are only considering the pickup time, as we will have some trips that begins late in one day and arrived on the next day, so I would merge these trips into the date when they started the trip)

In [20]:
# Create the count of each date in the data

data.groupby(pd.to_datetime(data['tpep_pickup_datetime']).dt.date).size()
# data['pickup_date_count'].head()

tpep_pickup_datetime
2021-01-01    24827
2021-01-02    34309
2021-01-03    26345
2021-01-04    44545
2021-01-05    46876
2021-01-06    49547
2021-01-07    50290
2021-01-08    50477
2021-01-09    39911
2021-01-10    29855
2021-01-11    46849
2021-01-12    50133
2021-01-13    51306
2021-01-14    53051
2021-01-15    53024
2021-01-16    38723
2021-01-17    31985
2021-01-18    37092
2021-01-19    51149
2021-01-20    49437
2021-01-21    53280
2021-01-22    54285
2021-01-23    42054
2021-01-24    31033
2021-01-25    47423
2021-01-26    47818
2021-01-27    52655
2021-01-28    56444
2021-01-29    54602
2021-01-30    39233
2021-01-31    31183
2021-02-01        3
2021-02-22        1
dtype: int64

In [21]:
# Step 1 - hard code the stuff
# style = 'seaborn'

# with plt.style.context(style):
#     fig, ax = plt.subplots(figsize=(10,5))
#     plt.bar(x=data[''])